In [18]:
import numpy as np
import sympy
from sympy import mod_inverse, isprime
import random
from math import gcd

# **The Rivest Shamir Adleman (RSA) Algorithm**

---

Public key cryptography is the concept of having a public key that is published and anyone can see or use it to send a message to someone, however only the person with the private key can decrypt the message. This eliminated having to send the decryption key over insecure channels which became more of a problem with the invention of computers.

### Table of Contents

- [1. Algorithm Overview](#algorithm-overview)
- [2. Mathematical Concepts Behind RSA](#mathematical-concepts-behind-rsa)
- [3. How the Algorithm Works](#how-the-algorithm-works)
- [4. Example of the RSA Algorithm](#example-of-the-rsa-algorithm)
- [5. RSA Algorithm with larger numbers](#example-of-the-rsa-algorithm-with-larger-numbers)
- [6. Finding the GCD of two numbers](#finding-the-gcd-of-two-numbers)
- [7. Finding the the coprime of a number](#finding-the-coprime-of-a-number)
- [8. Checking if a number is Prime with Fermat's Little Theorem](#checking-if-a-number-is-prime-with-fermats-little-theorem)
- [9. Understanding Modular Arithmetic](#understanding-modular-arithmetic)
- [10. Prime Factorisation](#prime-factorisation-and-its-role-in-rsa)
  
---

## **1. Algorithm Overview**

The **Rivest Shamir Adleman algorithm**, more commonly known as the **RSA algorithm**, was the first published example of a workable public key system created in 1977. The RSA Algorithm generates a public/private key pair by doing the following steps:

1. **Select two large prime numbers and call them `p` and `q`.** Note, this algorithm works well when these numbers are very large.
2. **Compute the modulus** `n` **by doing** `n = p * q` **then compute the Euler’s totient function to find** `φ(pq) = (p - 1)(q - 1)`
3. **Select at random the encryption key** `e` **that meets** `1 < e < φ(pq)` **where the gcd is** `gcd(e, φ(pq)) = 1`
4. **Then you solve the following equation so that you can find the decryption key** `d`:

`d * e ≡ 1 (mod φ(pq)), 0 ≤ d ≤ pq`

## **2. Mathematical Concepts Behind RSA**

To see some of the mathematical concepts behind the RSA algorithm, check out the [Number Theory Basics](../Number-Theory/Number-Theory-Basics.ipynb) file. Here you'll find explanations and code examples for the following concepts:
- **Checking if a number is Prime with Fermat's Little Theorem**
- **Finding the coprime of a number**
- **Finding the Greatest Common Divisor (GCD) and Least Common Divisor (LCM) of two numbers**
- **Prime Factorisation**

## **3. How the Algorithm Works**
- To use the algorithm, each user will publish their public encryption key along with the modulus `n` which is shown as `PU = {e, n}`. 
- They keep the secret decryption key along with the primes used `PR = {d, p, q}`. 
- To encrypt a message `M`, the sender will obtain the public key of the recipient, compute the ciphertext `c` to `c = M^e (mod n)` where `0 ≤ M ≤ n` and then sends the ciphertext `C` to the recipient. 
- In reverse, the decryption of ciphertext `C` by the recipient is done by using their private decryption key `d` and computing the message `M = C^d (mod n)`.

In the code you will notice the use of `%%time`, this is a magic command in Jupyter Notebook that will time how long it takes to run the code. This is useful to see how long it takes to generate the keys and encrypt/decrypt a message. At first you'll see that it's incredibly quick in the inital example, but as you increase the size of the prime numbers, you'll see that it takes longer to generate the keys and encrypt/decrypt a message.

## **4. Example of the RSA Algorithm**

---

### 1. Function Generation
The Euler's Totient function is calculated as follows:

- If `n` is a prime number, then `φ(n) = n - 1`
- If `n` is a product of two prime numbers, then `φ(n) = (p - 1)(q - 1)`
- If `n` is a product of three prime numbers, then `φ(n) = (p - 1)(q - 1)(r - 1)`
- And so on...

In [105]:
def euler_phi(n):
    result = n
    p = 2
    while p * p <= n:
        if n % p == 0:
            while n % p == 0:
                n //= p
            result -= result // p
        p += 1
    if n > 1:
        result -= result // n
    return result

def power_mod(base, exp, mod):
    return pow(base, exp, mod)

We will define the following functions:
- `find_private_key`: To find the private key `d` using the public key exponent `e` and the totient of `n`
- `encrypt_message`: To encrypt the message using the public key exponent `e` and the modulus `n`
- `decrypt_message`: To decrypt the message using the private key `d` and the modulus `n`

In [104]:
# def power_mod(base, exp, mod):
def find_private_key(e, totient_n):
    if gcd(e, totient_n) != 1:
        raise ValueError("e and totient_n are not coprime")
    return power_mod(e, -1, totient_n)

# def find_private_key(e, totient_n):
def encrypt_message(message, e, n):
    return power_mod(message, e, n)

# def encrypt_message(message, e, n):
def decrypt_message(ciphertext, d, n):
    return power_mod(ciphertext, d, n)

### 2. Define the variables

Let's define the variables for the RSA algorithm:

- `p` and `q`: Two large prime numbers
- `n`: The product of two large prime numbers
- `e`: The public key exponent

We will calculate the totient of `n` using the Euler's Totient function.

In [58]:
p = 97
q = 101

In [59]:
%%time
n = p * q
e = 17
totient_n = euler_phi(n)
print("Totient of n:", totient_n)

Totient of n: 9600
CPU times: user 103 μs, sys: 0 ns, total: 103 μs
Wall time: 94.7 μs


### 3. Find the private key `d`

We will find the private key `d` using the public key exponent `e` and the totient of `n`.

This is done by solving the equation `d * e ≡ 1 (mod φ(pq))` where `0 ≤ d ≤ pq`.

In [19]:
%%time
d = find_private_key(e, totient_n)
print("Private key d:", d)

Private key d: 3953
CPU times: user 41 μs, sys: 58 μs, total: 99 μs
Wall time: 92 μs


### 4. Encrypt the message

We will encrypt the message `1234` using the public key exponent `e` and the modulus `n`.

The ciphertext is calculated as `c = M^e (mod n)`.

In [20]:
%%time
message = 1234
ciphertext = encrypt_message(message, e, n)
print("Original message:", message)
print("Ciphertext:", ciphertext)

Original message: 1234
Ciphertext: 8897
CPU times: user 49 μs, sys: 69 μs, total: 118 μs
Wall time: 110 μs


### 5. Decrypt the message

We will decrypt the ciphertext using the private key `d` and the modulus `n`.

The message is calculated as `M = C^d (mod n)`.

The decrypted message should be the same as the original message.

In [21]:
%%time
decrypted_message = decrypt_message(ciphertext, d, n)
print("Decrypted message:", decrypted_message)
print(f"Is the decrypted message the same as the original message? {message == decrypted_message}")

Decrypted message: 1234
Is the decrypted message the same as the original message? True
CPU times: user 126 μs, sys: 0 ns, total: 126 μs
Wall time: 121 μs


### 6. Putting it all together

We will put all the steps together to show how the RSA algorithm works.

In [9]:
print(f"Our original value of n was: {n}")
print(f"Our original value of e was: {e}")
print(f"Our calculated value of totient_n was: {totient_n}")
print(f"Our original message was: {message}")
print(f"Our encrypted message was: {ciphertext}")
print(f"Our decrypted message was: {decrypted_message}")
print(f"Is the decrypted message the same as the original message? {message == decrypted_message}")

Our original value of n was: 9797
Our original value of e was: 17
Our calculated value of totient_n was: 9600
Our original message was: 1234
Our encrypted message was: 8897
Our decrypted message was: 1234
Is the decrypted message the same as the original message? True


## **5. Let's try the RSA Algorithm on some Larger Numbers**

---

Here we can condense down our code blocks as we have initially declared a lot of our functions and variables. We will now try the RSA algorithm on some larger numbers to see how it works.

We will also definte a prime number generator function `generate_prime_number` to generate a prime number of a given size.

This will be useful later when testing larger values of `p` and `q`.

A handy way of dealing with larger numbers and trying to find the `tiotient` of `n` is to use the `math` library in Python. This library has a function called `math.gcd` which can be used to find the greatest common divisor of two numbers. This is useful when trying to find the `tiotient` of `n` as we can use the formula `gcd(e, φ(pq)) = 1`.

### 1. Generate a Prime Number Function

In [115]:
def generate_prime(start=100000, end=800000):
    while True:
        num = random.randint(start, end)
        if sympy.isprime(num):
            return num
        
def print_rsa_values(p, q, n, e, totient_n, message, ciphertext, decrypted_message):
    print(f"Our original value of p and q were: {p} and {q}")
    print(f"Our original value of n was: {n}")
    print(f"Our original value of e was: {e}")
    print(f"Our calculated value of totient_n was: {totient_n}")
    print(f"Our original message was: {message}")
    print(f"Our encrypted message was: {ciphertext}")
    print(f"Our decrypted message was: {decrypted_message}")
    print(f"Is the decrypted message the same as the original message? {message == decrypted_message}")

---

### 2. The RSA algorithm on 5 digit prime numbers

Here the total time for generating the keys and encrypting/decrypting a message is shown. You can see that it takes a bit longer to generate the keys and encrypt/decrypt a message when using 5 digit prime numbers. With it being `40.1 μs` for the entire process.

In [124]:
%%time
p = 10427
q = 10663
n = p * q
e = 17
message = 1234

totient_n = euler_phi(p) * euler_phi(q)
while gcd(e, totient_n) != 1:
    e += 2  # Increment e until it's coprime with totient_n
    
d = find_private_key(e, totient_n)
ciphertext = encrypt_message(message, e, n)
decrypted_message = decrypt_message(ciphertext, d, n)

CPU times: user 36 μs, sys: 0 ns, total: 36 μs
Wall time: 40.5 μs


In [126]:
print_rsa_values(p, q, n, e, totient_n, message, ciphertext, decrypted_message)

Our original value of p and q were: 10427 and 10663
Our original value of n was: 111183101
Our original value of e was: 17
Our calculated value of totient_n was: 111162012
Our original message was: 1234
Our encrypted message was: 104710521
Our decrypted message was: 1234
Is the decrypted message the same as the original message? True


---

### 3. The RSA algorithm on 10 digit prime numbers

Here the total time for generating the keys and encrypting/decrypting a message is shown. You can see that it takes a bit longer to generate the keys and encrypt/decrypt a message when using 10 digit prime numbers. With it now being `26.5 ms` for the entire process, a significant increase from the 5 digit prime numbers.

In [96]:
%%time

p = generate_prime(1000000000, 8000000000)
q = generate_prime(1000000000, 8000000000)
n = p * q
e = 17
message = 123456789

totient_n = euler_phi(p) * euler_phi(q)
while gcd(e, totient_n) != 1:
    e += 2  # Increment e until it's coprime with totient_n

d = find_private_key(e, totient_n)
ciphertext = encrypt_message(message, e, n)
decrypted_message = decrypt_message(ciphertext, d, n)

CPU times: user 27.2 ms, sys: 0 ns, total: 27.2 ms
Wall time: 26.5 ms


In [80]:
print_rsa_values(p, q, n, e, totient_n, message, ciphertext, decrypted_message)

Our original value of p and q were: 37201553 and 14549159
Our original value of n was: 541251309643927
Our original value of e was: 17
Our calculated value of totient_n was: 541251257893216
Our original message was: 123456789
Our encrypted message was: 514680304925327
Our decrypted message was: 123456789
Is the decrypted message the same as the original message? True


---

### 4. The RSA algorithm on 15 digit prime numbers

Here the total time for generating the keys and encrypting/decrypting a message is shown. You can see that it takes longer to generate the keys and encrypt/decrypt a message when using 15 digit prime numbers. With it now being `4.7 s` for the entire process, which is a significant jump from 10 digit prime numbers and even more so from 5 digit prime numbers.

In [108]:
%%time

p = generate_prime(100000000000000, 900000000000000)
q = generate_prime(100000000000000, 900000000000000)
n = p * q
e = 17
message = 123456789

totient_n = euler_phi(p) * euler_phi(q)
while gcd(e, totient_n) != 1:
    e += 2  # Increment e until it's coprime with totient_n

d = find_private_key(e, totient_n)
ciphertext = encrypt_message(message, e, n)
decrypted_message = decrypt_message(ciphertext, d, n)

CPU times: user 4.7 s, sys: 2 ms, total: 4.7 s
Wall time: 4.71 s


In [90]:
print_rsa_values(p, q, n, e, totient_n, message, ciphertext, decrypted_message)

Our original value of p and q were: 817140172394107 and 547754557755289
Our original value of n was: 447592253753814692862971681923
Our original value of e was: 17
Our calculated value of totient_n was: 447592253753813327968241532528
Our original message was: 123456789
Our encrypted message was: 266981944652092286561115982209
Our decrypted message was: 123456789
Is the decrypted message the same as the original message? True


---

### 5. The RSA algorithm on 18 digit prime numbers

Here the total time for generating the keys and encrypting/decrypting a message is shown. You can see that it takes longer to generate the keys and encrypt/decrypt a message when using 18 digit prime numbers. With it now being `1 min 48 s` for the entire process, which is a significant jump from 15 digit prime numbers and even more so from 10 digit prime numbers.

In [111]:
%%time

p = generate_prime(100000000000000000, 900000000000000000)
q = generate_prime(100000000000000000, 900000000000000000)
n = p * q
e = 17
message = 123456789

totient_n = euler_phi(p) * euler_phi(q)
while gcd(e, totient_n) != 1:
    e += 2  # Increment e until it's coprime with totient_n
    
d = find_private_key(e, totient_n)
ciphertext = encrypt_message(message, e, n)
decrypted_message = decrypt_message(ciphertext, d, n)

CPU times: user 1min 48s, sys: 997 μs, total: 1min 48s
Wall time: 1min 48s


In [112]:
print_rsa_values(p, q, n, e, totient_n, message, ciphertext, decrypted_message)

Our original value of p and q were: 415573428518459957 and 543966113095135777
Our original value of n was: 226057862616805912578094787852581589
Our original value of e was: 17
Our calculated value of totient_n was: 226057862616805911618555246238985856
Our original message was: 123456789
Our encrypted message was: 160107104791976571231752716075044555
Our decrypted message was: 123456789
Is the decrypted message the same as the original message? True


---

### 6. The RSA algorithm on 19 digit prime numbers

The final example shows the RSA algorithm on 19 digit prime numbers. Here the total time for generating the keys and encrypting/decrypting a message is shown. You can see that it takes longer to generate the keys and encrypt/decrypt a message when using 19 digit prime numbers. With it now being `10 min 13 s` for the entire process, which is a significant jump from 18 digit prime numbers and even more so from 15 digit prime numbers.

In [113]:
%%time

p = generate_prime(1000000000000000000, 9000000000000000000)
q = generate_prime(1000000000000000000, 9000000000000000000)
n = p * q
e = 17
message = 123456789

totient_n = euler_phi(p) * euler_phi(q)
while gcd(e, totient_n) != 1:
    e += 2  # Increment e until it's coprime with totient_n
    
d = find_private_key(e, totient_n)
ciphertext = encrypt_message(message, e, n)
decrypted_message = decrypt_message(ciphertext, d, n)

CPU times: user 10min 12s, sys: 92 ms, total: 10min 13s
Wall time: 10min 13s


In [114]:
print_rsa_values(p, q, n, e, totient_n, message, ciphertext, decrypted_message)

Our original value of p and q were: 5187424496365756861 and 6896938881277935473
Our original value of n was: 35777349702678600972672615955165030253
Our original value of e was: 17
Our calculated value of totient_n was: 35777349702678600960588252577521337920
Our original message was: 123456789
Our encrypted message was: 2177553986524605235606780047389748843
Our decrypted message was: 123456789
Is the decrypted message the same as the original message? True


So as you can see, the RSA algorithm is incredibly powerful and secure, however, it can be slow to compute when using large prime numbers. This is why it's important to use large prime numbers to ensure the security of the algorithm. The RSA algorithm often uses numbers that are hundreds of digits long to ensure that it's secure.

Although the RSA algorithm is secure, it's important to note that it can be vulnerable to attacks such as the **timing attack** and the **low exponent attack**. This is why it's important to use large prime numbers and to ensure that the public and private keys are kept secure.